In [1]:
import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_stata("callreports_final.dta", chunksize=100000, preserve_dtypes=True)

df_raw = pd.DataFrame()

for chunk in data:
    df_raw=df_raw.append(chunk)

In [5]:
%matplotlib inline
import numpy as np
import seaborn as sns
import statsmodels.api as sm
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
from matplotlib import ticker

import cpi

pd.set_option('mode.chained_assignment', None)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

SMALL_SIZE = 8
MEDIUM_SIZE = 10
BIGGER_SIZE = 18

plt.style.use('default')


plt.rcParams['font.size']=BIGGER_SIZE          # controls default text sizes
plt.rcParams['axes.titlesize']=BIGGER_SIZE     # fontsize of the axes title
plt.rcParams['axes.labelsize']=BIGGER_SIZE    # fontsize of the x and y labels
plt.rcParams['xtick.labelsize']=BIGGER_SIZE    # fontsize of the tick labels
plt.rcParams['ytick.labelsize']=BIGGER_SIZE    # fontsize of the tick labels
plt.rcParams['legend.fontsize']=BIGGER_SIZE    # legend fontsize
plt.rcParams['figure.titlesize']=BIGGER_SIZE  # fontsize of the figure title
plt.rcParams['xtick.bottom']=True  # fontsize of the figure title
plt.rcParams['xtick.major.bottom']=True  # fontsize of the figure title
plt.rcParams['xtick.major.size']=3.5  # fontsize of the figure title
#plt.rcParams['xtick.major.width']=1  # fontsize of the figure title
plt.rcParams['xtick.major.top']=False  # fontsize of the figure title
plt.rcParams['xtick.minor.bottom']=True  # fontsize of the figure title
plt.rcParams['xtick.minor.size']=3  # fontsize of the figure title

#Grid
plt.rcParams['grid.color'] = 'k'
plt.rcParams['grid.linestyle'] = ':'
plt.rcParams['grid.linewidth'] = 0.5


from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

#plt.rcParams['legend.fontsize'] = 22

In [25]:
df = df_raw[["rssdid", "name", 'year', "quarter","assets", "equity","loans" ,"persloans", "agloans", "reloans", "ciloans"]]

In [59]:
df.year = df.year.astype("int64")
df.rssdid = df.year.astype("int64")
df.rssdid = df.quarter.astype("int64")

df.assets.dtype

df.iloc[:,4:]

assets    equity     loans  persloans  agloans   reloans   ciloans  assets_adjusted
0          2779.0    1991.0      23.0       23.0      0.0       0.0       0.0      6367.028768
1          4322.0    1977.0    1912.0      820.0      0.0     808.0     284.0      9902.230420
2          5118.0    1968.0    2983.0     1063.0      0.0    1406.0     509.0     11725.963741
3          5742.0    1956.0    3429.0      995.0      0.0    1701.0     717.0     13155.624033
4          7737.0    1942.0    4704.0     1179.0      0.0    2451.0    1038.0     17102.243477
...           ...       ...       ...        ...      ...       ...       ...              ...
1609535  770830.0  170330.0  404373.0      173.0      0.0   81532.0  303066.0    830886.424576
1609536  808999.0  168617.0  496526.0      175.0      0.0   82185.0  394437.0    872029.223818
1609537   35356.0   15989.0    5652.0        0.0    126.0    3517.0    2009.0     38110.634546
1609538  507242.0   51487.0  319536.0     2233.0      0.0  269612.0   47691.0    546761.921273
1609539  270022.0  239316.0       0.0        0.0      0.0       0.0       0.0    291059.785085

[1609540 rows x 8 columns]

In [60]:
from tqdm import tqdm, tqdm_notebook
tqdm_notebook().pandas()

for column in df.iloc[:,4:]:
        print(column)
        df[column] = df.progress_apply(lambda x: cpi.inflate(x[column], x.year), axis=1)

#df["assets_adjusted"] = df.apply(lambda x: cpi.inflate(x.columns, x.year), axis=1)



C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


assets


C:\ProgramData\Anaconda3\lib\site-packages\tqdm\std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel



equity



loans



persloans



agloans



reloans



ciloans



assets_adjusted


In [46]:
#aggregate per year
df_agg = df.groupby(["year", "quarter"]).sum()

#create index
index = pd.Index(sm.tsa.datetools.dates_from_range('1976Q1','2013Q4'))

print(index.dtype)
df_agg.index = index

df_agg.plot(y=["assets","assets_adjusted"], kind="line", figsize=(20,10))





datetime64[ns]


dtype('<M8[ns]')

ValueError: unsupported format character 'y' (0x79) at index 2

<Figure size 2000x1000 with 1 Axes>